In [15]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os.path
import mediapipe as mp
import time
import os
import cv2.aruco as aruco

h=720
w=1280
windowscale=0.6

pc = rs.pointcloud()

fps=30
try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, r'C:\Users\arpan\OneDrive\Documents\internship\bags\gpane.bag', repeat_playback=False)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, fps)
    config.enable_stream(rs.stream.color, rs.format.rgb8, fps)

    # Start streaming from file
    profile=pipeline.start(config)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    #Needed so frames don't get dropped during processing:
    profile.get_device().as_playback().set_real_time(False)

    align_to = rs.stream.color
    align = rs.align(align_to)

    timestamps=[]
    
    # Frame at which to find the L frame
    calib_frame=50

    # Frame counter
    c=0

    # Streaming loop
    while True:

        frame_present, frameset = pipeline.try_wait_for_frames()
    
        #End loop once video finishes
        if not frame_present:
            break

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # get timestamp of frame
        timestamps.append(rs.frame.get_frame_metadata(depth_frame,rs.frame_metadata_value.time_of_arrival))

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() 
        color_frame = aligned_frames.get_color_frame()

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data()) 
        color_image = np.asanyarray(color_frame.get_data()) 

        #converting color image to BGR
        color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images =np.hstack((color_image, depth_colormap))

        
        # When calibration frame is reached find the 3D position data and save it in xyzpos
        if c==calib_frame:
            mapped_frame = color_frame
            pc.map_to(mapped_frame)
            
            try:
                points = pc.calculate(aligned_depth_frame)
                v = points.get_vertices()
                verts = np.asanyarray(v).view(np.float32)
                xyzpos=verts.reshape(h,w, 3)  # xyz     
            except:
                print(type(v))
            break
        # print(c)
        c+=1

finally:
    # Specify the ArUco dictionary
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)

    # Create the parameters for the ArUco detector
    parameters = aruco.DetectorParameters_create()

    # Detect the ArUco markers in the image
    corners, ids, _ = aruco.detectMarkers(color_image, aruco_dict, parameters=parameters)

    for corner in corners:
            center = np.mean(corner[0], axis=0)
            x = int(center[0])
            y = int(center[1])
            # cv2.circle(color_image, (x, y), 50, (0, 255, 0), 2)

    # Draw a circle around the center of each detected marker
    centers=[]
    for corner in corners:
            center = np.mean(corner[0], axis=0)
            x = int(center[0])
            y = int(center[1])
            centers.append((x,y))
            # cv2.circle(color_image, (x, y), 50, (0, 255, 0), 2)

    cv2.circle(color_image, (centers[0][0],centers[0][1]), 30, (255, 0, 0), 4) #blue 
    cv2.circle(color_image, (centers[1][0],centers[1][1]), 30, (0, 255, 0), 4) #green
    cv2.circle(color_image, (centers[2][0],centers[2][1]), 30, (0, 0, 255), 4) #red
    cv2.imshow("show", color_image)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        pass
    try:
        if (color_image)==-1:
            cv2.destroyAllWindows()
            pass
    except:
        pass

    cv2.waitKey(0)
    # Show images
    cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RealSense', color_image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        pass

    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    print(c)

50


In [16]:
# Assigning centers to origin, x axis and z axis
for i in range(3):
    for j in range(3):
        if i!=j:
            if 19<np.linalg.norm(xyzpos[centers[i][1],centers[i][0]]-xyzpos[centers[j][1],centers[j][0]])*100<21 and 14<np.linalg.norm(xyzpos[centers[3-(j+i)][1],centers[3-(j+i)][0]]-xyzpos[centers[j][1],centers[j][0]])*100<16:
                cento=xyzpos[centers[j][1],centers[j][0]]
                centz=xyzpos[centers[i][1],centers[i][0]]
                centx=xyzpos[centers[3-(j+i)][1],centers[3-(j+i)][0]]
                print('Centers assigned')

Centers assigned


In [17]:
#verifiying centers
org_z=np.add(centz,-cento)*100
org_x=np.add(cento,-centx)*100
print(np.linalg.norm(org_z))
print(np.linalg.norm(org_x))

20.006002
14.814707


In [18]:
# Finding the Rotation matrix
v1 = centx - cento  # v1
v2 = centz - cento  # v2

vxnorm = v1 / np.linalg.norm(v1)
print(vxnorm)
vzcap = v2 - (vxnorm.T @ v2) * vxnorm
print(vzcap)
vznorm = vzcap / np.linalg.norm(vzcap)
print(vznorm)

vynorm=np.cross(vznorm.T,vxnorm.T).reshape(3,1)
print(vynorm)

print(np.linalg.norm(vynorm))

vznorm=vznorm.reshape(3,1)
vxnorm=vxnorm.reshape(3,1)

rotMat = np.hstack((vxnorm, vynorm, vznorm))

[ 0.9999033  -0.00333314 -0.01349992]
[-0.00257728  0.02846707 -0.19792107]
[-0.01288807  0.14235358 -0.9897319 ]
[[-0.00522068]
 [-0.9898102 ]
 [-0.14229685]]
0.99999994


In [19]:
print(rotMat)
print(cento)

[[ 0.9999033  -0.00522068 -0.01288807]
 [-0.00333314 -0.9898102   0.14235358]
 [-0.01349992 -0.14229685 -0.9897319 ]]
[0.13313153 0.33307818 1.3490001 ]


In [20]:
# Saving the roation Matrix and the origin to files
with open(r'D435_rotmat.txt', 'w') as fp:
    for item in rotMat:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'D435_org.txt', 'w') as fp:
    for item in cento:
        # write each item on a new line
        fp.write("%s\n" % item)